In [1]:
import pickle
import pandas as pd
import numpy as np
import optuna

#     optuna_ex1_hyperparameters_per_dataset.py
#  -> analysis_ex1_hyperparameters.ipynb
#  -> benchmark_ex1_best_hyperparameters.py
#  -> analysis_ex1_hyperparameters_best.ipynb

In [2]:
SETUP = "2D"  # "FG" or "2D"
storage = "sqlite:///optuna_databases/optuna_ex1_parameter_tuning_{}_e3_d2.sqlite3".format(SETUP)  

In [3]:
studies = optuna.study.get_all_study_summaries(storage=storage)

In [4]:
loaded_study = optuna.load_study(study_name=studies[0].study_name, storage=storage)
df = loaded_study.trials_dataframe()
for key, value in loaded_study.system_attrs.items():
    df[key] = value
for study in studies[1:]:
    loaded_study = optuna.load_study(study_name=study.study_name , storage=storage)
    df_new = loaded_study.trials_dataframe()
    for key, value in loaded_study.system_attrs.items():
        df_new[key] = value
    df = pd.concat([df, df_new])
df.reset_index(inplace=True, drop=True)

In [5]:
df.columns

Index(['number', 'value', 'datetime_start', 'datetime_complete', 'duration',
       'params_init_learning_rate', 'params_learning_rate_decay',
       'params_max_depth', 'params_n_experts', 'params_optimization_method',
       'params_use_2_dim_clustering', 'params_use_2_dim_gate_based_on',
       'user_attrs_accuracy_train', 'user_attrs_accuracy_val',
       'user_attrs_std_train', 'user_attrs_std_val', 'state', 'Data X',
       'Data y', 'initialization_method', 'params_alpha', 'params_beta',
       'params_gamma', 'params_mean_precision_prior',
       'params_weight_concentration_prior',
       'params_weight_concentration_prior_type', 'params_weight_cutoff'],
      dtype='object')

In [6]:
df.head(1)

,number,value,datetime_start,datetime_complete,duration,params_init_learning_rate,params_learning_rate_decay,params_max_depth,params_n_experts,params_optimization_method,...,Data X,Data y,initialization_method,params_alpha,params_beta,params_gamma,params_mean_precision_prior,params_weight_concentration_prior,params_weight_concentration_prior_type,params_weight_cutoff
0,0,0.717752,2021-08-18 20:51:08.789068,2021-08-18 20:51:16.361862,0 days 00:00:07.572794,187.948083,0.875969,2,3,ridge_regression,...,abalone_input.pd,abalone_target.pd,str,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df.initialization_method.value_counts()

Kmeans_init      9
str              9
BGM_init         9
KDTmeans_init    9
Name: initialization_method, dtype: int64

In [8]:
np.unique(df.initialization_method)

array(['BGM_init', 'KDTmeans_init', 'Kmeans_init', 'str'], dtype=object)

In [9]:
np.unique(df["Data X"])

array(['abalone_input.pd', 'banknote_input.pd', 'breast_cancer_input.np',
       'cars_input.pd', 'contraceptive_input.pd', 'generated6_input.np',
       'iris_input.pd', 'steel_input.pd', 'students_input.pd'],
      dtype=object)

In [10]:
top = 10
df_best = pd.DataFrame()
for dataset in np.unique(df["Data X"]):
    df_new = df[df["Data X"] == dataset].sort_values(by=["user_attrs_accuracy_val"], ascending=False).head(top)
    df_best = pd.concat([df_best, df_new])
df_best.reset_index(inplace=True, drop=True)


In [11]:
df_best.shape

(36, 27)

In [12]:
df_best.head(2)

,number,value,datetime_start,datetime_complete,duration,params_init_learning_rate,params_learning_rate_decay,params_max_depth,params_n_experts,params_optimization_method,...,Data X,Data y,initialization_method,params_alpha,params_beta,params_gamma,params_mean_precision_prior,params_weight_concentration_prior,params_weight_concentration_prior_type,params_weight_cutoff
0,0,0.717752,2021-08-18 20:51:08.789068,2021-08-18 20:51:16.361862,0 days 00:00:07.572794,187.948083,0.875969,2,3,ridge_regression,...,abalone_input.pd,abalone_target.pd,str,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0.699234,2021-08-18 20:51:33.398168,2021-08-18 20:51:41.674589,0 days 00:00:08.276421,63.032069,0.903005,2,3,ridge_regression,...,abalone_input.pd,abalone_target.pd,KDTmeans_init,4.545768,0.413105,0.301518,NaN,NaN,NaN,NaN


In [13]:
pickle.dump(df_best, open("dataframes/ex1_df_top10_hyperparameters_per_dataset_{}_e3_d2.pd".format(SETUP), "wb"))

In [16]:
df.groupby(["Data X"]).mean()

,number,value,params_init_learning_rate,params_learning_rate_decay,params_max_depth,params_n_experts,params_use_2_dim_clustering,user_attrs_accuracy_train,user_attrs_accuracy_val,user_attrs_std_train,user_attrs_std_val,params_alpha,params_beta,params_gamma,params_mean_precision_prior,params_weight_concentration_prior,params_weight_cutoff
Data X,,,,,,,,,,,,,,,,,
abalone_input.pd,0,0.693487,92.656662,0.868942,2,3,True,0.712857,0.693487,0.006852,0.011622,4.545768,0.413105,0.301518,0.407097,0.576575,0.0
banknote_input.pd,0,0.969629,170.874130,0.897690,2,3,True,0.982589,0.969629,0.006171,0.008855,9.056486,0.308975,0.398375,0.158154,0.840336,0.0
breast_cancer_input.np,0,0.916593,182.595867,0.874123,2,3,True,0.964591,0.916593,0.006544,0.024028,5.069897,0.438316,0.125604,0.916387,0.387578,0.0
cars_input.pd,0,0.804784,126.041138,0.918511,2,3,True,0.810828,0.804784,0.009865,0.019890,7.846580,0.012076,0.040711,0.329209,0.115238,0.0
contraceptive_input.pd,0,0.504076,184.253596,0.864794,2,3,True,0.539629,0.504076,0.008494,0.021908,4.020739,0.374685,0.142743,0.133544,0.788327,0.0
generated6_input.np,0,0.843534,153.548384,0.860894,2,3,True,0.848444,0.843534,0.013515,0.017166,0.596235,0.081598,0.258168,0.592674,0.230575,0.0
iris_input.pd,0,0.930804,173.669170,0.847065,2,3,True,0.988095,0.930804,0.007792,0.053902,8.844577,0.125508,0.206916,0.256682,0.308756,0.0
steel_input.pd,0,0.557735,206.638564,0.904618,2,3,True,0.586139,0.557735,0.014559,0.013924,1.463865,0.497185,0.207565,0.969861,0.427250,0.0
students_input.pd,0,0.474444,161.577293,0.883878,2,3,True,0.562966,0.474444,0.020129,0.035497,6.271474,0.140089,0.421821,0.444877,0.320664,0.0


In [15]:
df_best.groupby(["Data X"]).mean()

,number,value,params_init_learning_rate,params_learning_rate_decay,params_max_depth,params_n_experts,params_use_2_dim_clustering,user_attrs_accuracy_train,user_attrs_accuracy_val,user_attrs_std_train,user_attrs_std_val,params_alpha,params_beta,params_gamma,params_mean_precision_prior,params_weight_concentration_prior,params_weight_cutoff
Data X,,,,,,,,,,,,,,,,,
abalone_input.pd,0,0.693487,92.656662,0.868942,2,3,True,0.712857,0.693487,0.006852,0.011622,4.545768,0.413105,0.301518,0.407097,0.576575,0.0
banknote_input.pd,0,0.969629,170.874130,0.897690,2,3,True,0.982589,0.969629,0.006171,0.008855,9.056486,0.308975,0.398375,0.158154,0.840336,0.0
breast_cancer_input.np,0,0.916593,182.595867,0.874123,2,3,True,0.964591,0.916593,0.006544,0.024028,5.069897,0.438316,0.125604,0.916387,0.387578,0.0
cars_input.pd,0,0.804784,126.041138,0.918511,2,3,True,0.810828,0.804784,0.009865,0.019890,7.846580,0.012076,0.040711,0.329209,0.115238,0.0
contraceptive_input.pd,0,0.504076,184.253596,0.864794,2,3,True,0.539629,0.504076,0.008494,0.021908,4.020739,0.374685,0.142743,0.133544,0.788327,0.0
generated6_input.np,0,0.843534,153.548384,0.860894,2,3,True,0.848444,0.843534,0.013515,0.017166,0.596235,0.081598,0.258168,0.592674,0.230575,0.0
iris_input.pd,0,0.930804,173.669170,0.847065,2,3,True,0.988095,0.930804,0.007792,0.053902,8.844577,0.125508,0.206916,0.256682,0.308756,0.0
steel_input.pd,0,0.557735,206.638564,0.904618,2,3,True,0.586139,0.557735,0.014559,0.013924,1.463865,0.497185,0.207565,0.969861,0.427250,0.0
students_input.pd,0,0.474444,161.577293,0.883878,2,3,True,0.562966,0.474444,0.020129,0.035497,6.271474,0.140089,0.421821,0.444877,0.320664,0.0


In [15]:
df["params_use_2_dim_gate_based_on"].value_counts(dropna=False)

NaN    4800
Name: params_use_2_dim_gate_based_on, dtype: int64

In [16]:
df_best.groupby(["params_optimization_method"]).mean()

,number,value,params_init_learning_rate,params_learning_rate_decay,params_max_depth,params_n_experts,params_use_2_dim_clustering,user_attrs_accuracy_train,user_attrs_accuracy_val,user_attrs_std_train,user_attrs_std_val,params_alpha,params_beta,params_gamma,params_mean_precision_prior,params_weight_concentration_prior,params_weight_cutoff
params_optimization_method,,,,,,,,,,,,,,,,,
lasso_regression,63.846154,0.748208,86.129834,0.984515,2.0,3.0,False,0.756270,0.748208,0.004511,0.009171,0.824356,0.426253,0.129430,NaN,NaN,NaN
least_squares_linear_regression,51.750000,0.861607,94.054996,0.987352,2.0,3.0,False,0.878175,0.861607,0.004964,0.012220,3.172670,0.195441,0.267134,0.525082,0.541967,0.0
ridge_regression,60.694915,0.788677,92.608457,0.986571,2.0,3.0,False,0.812043,0.788677,0.007222,0.012849,2.690768,0.215031,0.264677,0.477155,0.583699,0.0


In [17]:
df_best.columns

Index(['number', 'value', 'datetime_start', 'datetime_complete', 'duration',
       'params_init_learning_rate', 'params_learning_rate_decay',
       'params_max_depth', 'params_n_experts', 'params_optimization_method',
       'params_use_2_dim_clustering', 'params_use_2_dim_gate_based_on',
       'user_attrs_accuracy_train', 'user_attrs_accuracy_val',
       'user_attrs_std_train', 'user_attrs_std_val', 'state', 'Data X',
       'Data y', 'initialization_method', 'params_alpha', 'params_beta',
       'params_gamma', 'params_mean_precision_prior',
       'params_weight_concentration_prior',
       'params_weight_concentration_prior_type', 'params_weight_cutoff'],
      dtype='object')

In [18]:
df_best["params_weight_concentration_prior_type"].value_counts()

dirichlet_process         14
dirichlet_distribution     5
Name: params_weight_concentration_prior_type, dtype: int64

In [19]:
df_best[df_best["params_weight_concentration_prior_type"] == "dirichlet_distribution"][['params_mean_precision_prior',
       'params_weight_concentration_prior']].mean()

params_mean_precision_prior          0.784972
params_weight_concentration_prior    0.714538
dtype: float64

In [20]:
df_best[df_best["params_weight_concentration_prior_type"] == "dirichlet_distribution"][['params_mean_precision_prior',
       'params_weight_concentration_prior']].median()

params_mean_precision_prior          0.808530
params_weight_concentration_prior    0.708875
dtype: float64

In [21]:
df_best[['params_alpha', 'params_beta','params_gamma']].mean()

params_alpha    2.673648
params_beta     0.228617
params_gamma    0.252616
dtype: float64

In [22]:
df_best[['params_alpha', 'params_beta','params_gamma']].median()

params_alpha    2.989458
params_beta     0.184299
params_gamma    0.248566
dtype: float64